In [1]:
CSV_TRAIN_FEATURE_PATH = "/home/ds/notebooks/datasets/UCI_HAR/Train/X_Train.txt"
CSV_TRAIN_LABEL_PATH = "/home/ds/notebooks/datasets/UCI_HAR/Train/Y_Train.txt"
CSV_TEST_FEATURE_PATH = "/home/ds/notebooks/datasets/UCI_HAR/Test/X_Test.txt"
CSV_TEST_LABEL_PATH = "/home/ds/notebooks/datasets/UCI_HAR/Test/Y_Test.txt"
CSV_ACTIVITY_LABEL_PATH = "/home/ds/notebooks/datasets/UCI_HAR/activity_labels.txt"
APP_NAME = "UCI HAR Random Forest Example"
SPARK_URL = "local[*]"
RANDOM_SEED = 141107
TRAINING_DATA_RATIO = 0.7
RF_NUM_TREES = 10
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pprint import pprint

In [3]:
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

activity_labels = spark.read.options(header = "true", inferschema = "true").csv(CSV_ACTIVITY_LABEL_PATH)

df_x_train = spark.read.text(CSV_TRAIN_FEATURE_PATH)
df_y_train = spark.read.text(CSV_TRAIN_LABEL_PATH)

df_x_test = spark.read.text(CSV_TEST_FEATURE_PATH)
df_y_test = spark.read.text(CSV_TEST_LABEL_PATH)

In [4]:
activity_labels.show()

+-----+------------------+
|index|             label|
+-----+------------------+
|    1|           WALKING|
|    2|  WALKING_UPSTAIRS|
|    3|WALKING_DOWNSTAIRS|
|    4|           SITTING|
|    5|          STANDING|
|    6|            LAYING|
+-----+------------------+



In [5]:
len(df_x_train.columns)

1

In [6]:
df_x_train.count()

7352

In [7]:
len(df_x_test.columns)

1